In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from pathlib import Path
from fla_utils import (
    PathLoader, 
    search, 
    REGRESSOR_COMBINATIONS,
    build_design_matrix,
    run_glm,
    extract_glm_metrics,
)

In [ ]:
# run parameters
test_flag = True # flag to run test
show_figures = False # flag to plot results in jupyter notebook
oscprep_dir = "/data/fastfmri_toolbox_test_data/7T/bids/derivatives/oscprep_grayords_fmapless"
base_out_dir = "/data/fastfmri_toolbox_test_data/7T/bids/derivatives/oscprep_grayords_fmapless_fla"
sub_id = '001'
ses_id = '02'
search_frequencies = [.2, .5]
time_window = (40, 14+185) # seconds

# Get paths for all task runs
subj_base_dir = f"{oscprep_dir}/bold_preproc/sub-{sub_id}/ses-{ses_id}/func"
task_runs = search(subj_base_dir, f"*task-[c,e]*_run-*_bold.nii.gz")
task_runs.sort()

# Loop over various design matrix schemes
for experiment_ix, dm_type in enumerate(REGRESSOR_COMBINATIONS.keys()):

    # Set-up directory
    out_dir=f"{base_out_dir}/{str(experiment_ix).zfill(2)}_experiment-{dm_type}"
    
    # Create figures directory
    figures_dir = Path(f"{out_dir}/NIFTI/figures")
    if not figures_dir.exists():
        figures_dir.mkdir(parents=True)
    
    # Loop over every task run
    for task_ix, task_run in enumerate(task_runs):
        
        # Exit when testing
        if test_flag and task_ix > 0:
            break

        # Extract info from each path
        stem = Path(task_run).stem
        task_id = stem.split('task-')[1].split('_')[0]
        run_id = stem.split('run-')[1].split('_')[0]
        print(f"[{str(task_ix+1).zfill(2)}/{str(len(task_runs)).zfill(2)}] [{dm_type}] sub-{sub_id}_ses-{ses_id}_task-{task_id}_run-{run_id}")

        # Skip if processed already
        processed_flag = Path(f"{out_dir}/NIFTI/sub-{sub_id}/ses-{ses_id}/task-{task_id}/run-{run_id}/GLM").exists() and Path(f"{out_dir}/CIFTI/sub-{sub_id}/ses-{ses_id}/task-{task_id}/run-{run_id}/GLM").exists()
        if processed_flag:
            print("Skipping.")
            continue

        # Instantiate path object
        path_loader = PathLoader(
            oscprep_dir,
            sub_id, 
            ses_id,
            task_id,
            run_id
        )

        # Build and save design matrix
        design_matrix, dm_fig = build_design_matrix(
            path_loader,
            time_window, 
            search_frequencies, 
            dm_type,
            show_flag = show_figures
        )
        dm_fig.savefig(f"{figures_dir}/sub-{path_loader.sub_id}_ses-{path_loader.ses_id}_task-{path_loader.task_id}_run-{path_loader.run_id}_DM.png")
        

        # Fit and run frequency-based GLM on NIFTI and CIFTI data
        for image_type in ['NIFTI','CIFTI']:
            run_glm(
                path_loader,
                time_window, 
                search_frequencies, 
                image_type, 
                design_matrix, 
                out_dir
            )
            # Visualize GLM results of NIFTI data
            if image_type == 'NIFTI':
                extract_glm_metrics(
                    path_loader,
                    f"{out_dir}/NIFTI",
                    figures_dir,
                    roi = "/data/fastfmri_toolbox_test_data/rois/roi-calcarine_space-MNI152NLin2009cAsym.nii.gz",
                    show_plots=show_figures,
                )